In [39]:
import sys
import networkx as nx
import matplotlib as plt
import numpy as np
import re
import chess
import subprocess
import timeit
import pickle
import stockfish
from stockfish import Stockfish
import chess
import chess.engine

engine = chess.engine.SimpleEngine.popen_uci("../stockfish-10-64")
stockfish = Stockfish("../stockfish-10-64")

In [40]:
def varWeighted(scores, counts):
    if (len(counts)<2):
        return None
    else:
        weightedMean = sum([a*b for a,b in zip(scores,counts)])/sum(counts)
        scoreDiffs = [(score - weightedMean)**2 for score in scores]
        V1 = sum(counts)
        V2 = sum([count**2 for count in counts])
        var = (V1/(V1**2-V2)) * sum([a*b for a,b in zip(counts,scoreDiffs)])
        return var
    
def set_score(san):
    board = chess.Board()
    for move in san:
        board.push_san(move)
    currFen = board.fen()
    score = engine.analyse(board, chess.engine.Limit(time=.05), info=chess.engine.INFO_SCORE)
    if('#' in str(score['score'])):
        if('-' in str(score['score'])):
            return -39765
        else:
            return 39765
    else:
        score = int(str(score["score"]))
    return score

def san_to_fen(san):
    board = chess.Board()
    for move in san:
        board.push_san(move)
    currFen = board.fen()
    return currFen

In [41]:
san_to_fen(['e4', 'e5', 'Nf3', 'Nc6', 'Bc4', 'Nf6', 'Ng5', 'd5', 'exd5', 'Nxd5', 'Nxf7', 'Kxf7', 'Qf3+', 'Ke6', 'Nc3', 'Ne7'])

'r1bq1b1r/ppp1n1pp/4k3/3np3/2B5/2N2Q2/PPPP1PPP/R1B1K2R w KQ - 4 9'

In [42]:
game_file = "../tab-trunc.txt"

with open(game_file, "r") as file_in:
    mainlineList = []
    for line in file_in:
        mainlineList.append(line)

#extract each string of moves
stringList = []
for mainLine in mainlineList:
    stringList.append(mainLine.split())

# stringList.sort(key = len) 


In [43]:
print(len(stringList))

692287


In [44]:
#Move ['32', 'f3', 'e5', 'g4', 'Qh4#']

board = chess.Board()
board.push_san('f3') #push the move to the board
board.push_san('e5') #push the move to the board
#board.push_san('g4') #push the move to the board
#board.push_san('Qh4#') #push the move to the board
currFen = board.fen()
print(currFen)
stockfish.set_fen_position(currFen)
score = stockfish.get_evaluation()['value']
print(score)

rnbqkbnr/pppp1ppp/8/4p3/8/5P2/PPPPP1PP/RNBQKBNR w KQkq - 0 2
-101


In [45]:
cnt = 0
start = timeit.default_timer()

#initialize graph with root node
g = nx.DiGraph()
g.add_node('root', score = 0, movelist = 'root')

for s in stringList[:2000]:
    whiteMove = 1
    if(cnt%1000 == 0):
        print("strings processed:", cnt, "TIME:", timeit.default_timer()-start)
    count = int(s[0])
    board = chess.Board()
    parentFen = 'root'
    try:
#         if(len(s) == 2):
#             board.push_san(s[1])
#             currFen = board.fen()
#             score = engine.analyse(board, chess.engine.Limit(time=0.05))
#             stockfish.set_fen_position(currFen)
#             score = int(str(score["score"]))
#             score = stockfish.get_evaluation()['value']
#             g.add_node(currFen,
#                     score = score,
#                     count = count,
#                     movelistCount = {parentFen:count})
#             g.add_edge(parentFen, currFen)
#         else:
            for move in s[1:]:
                board.push_san(move)
                currFen = board.fen()
                if(currFen not in g.nodes):
                    board = chess.Board(currFen)
                    if('#' not in move):
                        score = engine.analyse(board, chess.engine.Limit(time=0.05))
                        if('#' in str(score['score'])):
                            if('-' in str(score['score'])):
                                score =  -39765
                            else:
                                score =  39765
                        else:
                            score = int(str(score["score"])) * whiteMove
#                         stockfish.set_fen_position(currFen)
#                         score = stockfish.get_evaluation()['value']
                    else:
                        score = 39765 * whiteMove
                    g.add_node(currFen,
                               score = score,
                               count = count,
                               movelistCount = {parentFen:count})
                    g.add_edge(parentFen, currFen)
                else:
                    if(parentFen in nx.get_node_attributes(g, 'movelistCount')[currFen]):
                        nx.get_node_attributes(g, 'movelistCount')[currFen][parentFen] += count #increase parent count
                    else:
                        nx.get_node_attributes(g, 'movelistCount')[currFen][parentFen] = count #add parent and count
                        g.add_edge(parentFen, currFen) #add edge from parentFen to currFen
                    g.nodes[currFen]['count'] = nx.get_node_attributes(g, 'count')[currFen] + count #increase count
#                 whiteMove *= -1
                parentFen = currFen
    except Exception as e:
        print(e)
        print(sys.exc_info()[0])
        print("Move",s)
        stockfish.set_fen_position(currFen)
        print(stockfish.get_evaluation()['value'])
        break
    cnt+=1
            
stop = timeit.default_timer()
print('Time: ', stop - start)
print("totalNodes:",len(g))

strings processed: 0 TIME: 0.0010037840002041776
strings processed: 1000 TIME: 126.96641232000002
Time:  336.802527283
totalNodes: 5345


In [ ]:
g.nodes()['rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2']

In [ ]:
#nx.write_gpickle(g,"test.gpickle")

In [8]:
g = nx.read_gpickle('g10k.gpickle')

In [21]:
start = timeit.default_timer()

board = chess.Board()
board.push_san('d4') #push the move to the board
board.push_san('Nf6') #push the move to the board
currFen = board.fen()
print(currFen)
scores = []
counts = []
for neighbor in list(g.neighbors(currFen)):
    scores.append(nx.get_node_attributes(g, 'score')[neighbor])
    counts.append(nx.get_node_attributes(g, 'count')[neighbor])
    
print("sd:",np.sqrt(varWeighted(scores, counts)))

stop = timeit.default_timer()
print('Time: ', stop - start) 

rnbqkb1r/pppppppp/5n2/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 1 2
sd: 38.691368920941464
Time:  0.026040993999913553


In [22]:
root = ['e4', 'c5']
safe = ['e4', 'c5', 'c3']
risky = ['e4', 'c5', 'd4']

root2 = ['e4', 'e6', 'd4', 'd5']
popular2 = ['e4', 'e6', 'd4', 'd5', 'Nc3']
popular21 = ['e4', 'e6', 'd4', 'd5', 'Nd2']
safe2 = ['e4', 'e6', 'd4', 'd5', 'exd5']
risky2 = ['e4', 'e6', 'd4', 'd5', 'e5']

Ruy = ['e4', 'e5','Nf3', 'Nc6','Bb5']
Vienna = ['e4' ,'e5','d4','exd4','c3']
games = [root,safe,risky,root2,popular2, popular21,safe2,risky2, Ruy, Vienna]
for game in games:
    board = chess.Board()
    for move in game:
        board.push_san(move)
    currFen = board.fen()
    scores = []
    counts = []
    for neighbor in list(g.neighbors(currFen)):
        scores.append(nx.get_node_attributes(g, 'score')[neighbor])
        counts.append(nx.get_node_attributes(g, 'count')[neighbor])

    print('{} ||| SD :'.format(game), np.sqrt(varWeighted(scores,counts)), ", total Games:", sum(counts))

['e4', 'c5'] ||| SD : 40.91255078328056 , total Games: 4070
['e4', 'c5', 'c3'] ||| SD : 36.76955262170047 , total Games: 47
['e4', 'c5', 'd4'] ||| SD : 103.9446968344225 , total Games: 234


TypeError: loop of ufunc does not support argument 0 of type NoneType which has no callable sqrt method

In [13]:
root = ['e4', 'c5']
safe = ['e4', 'c5', 'c3']
risky = ['e4', 'c5', 'd4']

root2 = ['e4', 'e6', 'd4', 'd5']
popular2 = ['e4', 'e6', 'd4', 'd5', 'Nc3']
popular21 = ['e4', 'e6', 'd4', 'd5', 'Nd2']
safe2 = ['e4', 'e6', 'd4', 'd5', 'exd5']
risky2 = ['e4', 'e6', 'd4', 'd5', 'e5']

Ruy = ['e4', 'e5','Nf3', 'Nc6','Bb5']
Vienna = ['e4' ,'e5','d4','exd4','c3']
games = [root,safe,risky,root2,popular2, popular21,safe2,risky2, Ruy, Vienna]


for game in games:
    board = chess.Board()
    for move in game:
        board.push_san(move)
    currFen = board.fen()
    stockfish.set_fen_position(currFen)
    info = stockfish.get_evaluation()['value']

    print(game,"Score:", info)

BrokenPipeError: [Errno 32] Broken pipe

In [14]:
pos = stockfish.set_fen_position("rnbqkbnr/pp1ppppp/8/2p5/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2")
out = engine.analyse(pos,chess.engine.Limit(time=0.1))
print(out["score"])

BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
stockfish.set_fen_position("rnbqkbnr/pp1ppppp/8/2p5/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2")
stockfish.get_evaluation()['value']

In [ ]:
#Deprecated because parent nodes not added
# cnt = 0

# start = timeit.default_timer()

# g = nx.DiGraph()
# g.add_node('', score = 0, movelist = 'root')
# for s in stringList:
#     if(cnt%1000 == 0):
#         print("strings processed:", cnt)
#     count = int(s[0])
#     board = chess.Board()
#     parentFen = 'root'
#     try:
#         if(len(s) == 2):
#             board.push_san(s[1])
#             currFen = board.fen()
#     #             score = engine.analyse(board, chess.engine.Limit(time=0.05))
#             stockfish.set_fen_position(currFen)
#     #             score = int(str(score["score"]))
#             score = stockfish.get_evaluation()['value']
#             g.add_node(currFen,
#                     score = score,
#                     count = count,
#                     movelistCount = {parentFen:count})
#         else:
#             for move in s[1:-1]:
#                 board.push_san(move)
#                 currFen = board.fen()
#                 if(currFen not in g.nodes):
#                     g.add_node(currFen,
#                                score = score,
#                                count = count,
#                                movelistCount = {})
#                     g.add_edge(parentFen, currFen)
#                 else:
#                     nx.get_node_attributes(g, 'movelistCount')[currFen][parentFen] = count
#                     g.nodes[currFen]['count'] = nx.get_node_attributes(g, 'count')[currFen] + count
#                 parentFen = board.fen()
#             board.push_san(s[-1])
#             currFen = board.fen()
#             if(currFen not in g.nodes):
#                 board = chess.Board(currFen)
#                 if('#' not in s[-1]):
#                     stockfish.set_fen_position(currFen)
#                     score = stockfish.get_evaluation()['value']
#                 else:
#                     score = 39765
#                 if(parentFen not in g.nodes):
#                     print(s, "no parent")
#                     g.add_node(currFen,
#                                score = score,
#                                count = count,
#                                movelistCount = {})
#                 else:
#                     g.add_node(currFen,
#                                score = score,
#                                count = count + nx.get_node_attributes(g, 'count')[parentFen],
#                                movelistCount = {parentFen:count})
#                 g.add_edge(parentFen, currFen)
#             else:
#                 nx.get_node_attributes(g, 'movelistCount')[currFen][parentFen] = count
#                 g.nodes[currFen]['count'] = nx.get_node_attributes(g, 'count')[currFen] + count
#     except Exception as e:
#         print(e)
#         print(sys.exc_info()[0])
#         print("Move",s)
#         stockfish.set_fen_position(currFen)
#         print(stockfish.get_evaluation()['value'])
#         break
#     cnt+=1
            
# stop = timeit.default_timer()
# print('Time: ', stop - start)
# print("totalNodes:",len(g))